<a href="https://colab.research.google.com/github/priscilabgmoya/GestorDeInspecciones/blob/main/redes_neuronales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RED NEURONAL - TRANSFORMAR DE C° A F°

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
celsius = np.array([-40,-10,0,8,15,22,38] , dtype= float)
fahrenhenit = np.array([-40,14,32,46,59,72,100],dtype= float)

In [19]:
#capa = tf.keras.layers.Dense(units=1, input_shape=[1])
#modelo = tf.keras.Sequential([capa])

#Que pasa si se agregan mas capas?
oculta_1 =  tf.keras.layers.Dense(units=3, input_shape=[1])
oculta_2 =  tf.keras.layers.Dense(units=3)
salida =  tf.keras.layers.Dense(units=1)
modelo = tf.keras.Sequential([oculta_1,oculta_2,salida])

In [20]:
modelo.compile(
    optimizer= tf.keras.optimizers.Adam(0.1),
    loss ='mean_squared_error'
)

In [ ]:
print("comenzamos entrenamiento")
historial = modelo.fit(celsius, fahrenhenit, epochs=100, verbose=False)
print("Modelo entrenado!")

In [ ]:
plt.xlabel("# Epoca")
plt.ylabel("Magnitud de Perdida")
plt.plot(historial.history["loss"])

In [ ]:
print("Hagamos una prediccion!")
resultado = modelo.predict([100.0])
print("El resultado es" + str(resultado) + "F")

In [ ]:
print("Variable Interna del modelo")
#print(capa.get_weights())
print(oculta_1.get_weights())
print(oculta_2.get_weights())
print(salida.get_weights())

In [ ]:
#Exportar el modelo en formato h5
modelo.save('celsius_a_fahrenheit.h5')

In [ ]:
!pip install tensorflowjs

In [61]:
!mkdir carpeta_salida

In [ ]:
!tensorflowjs_converter  --input_format keras celsius_a_fahrenheit.h5  carpeta_salida

# RED NEURONAL - CLASIFICADOR DE IMAGENES

In [56]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import math
import numpy as np

In [ ]:
datos, metadatos = tfds.load('fashion_mnist', as_supervised=True, with_info=True)

In [33]:
datos_entrenamiento , datos_prueba = datos['train'], datos['test']
nombres_clases = metadatos.features['label'].names

In [ ]:
#normalizar los datos (pasar de 0-255 a 0-1)
def normalizar(imagenes,etiquetas):
  imagenes = tf.cast(imagenes, tf.float32)
  imagenes /= 255 #aqui lo pasa de 0-255 a 0-1
  return imagenes, etiquetas

#normalizamos los datos de entrenamiento y prueba con la funcion que hicimos
datos_entrenamiento = datos_entrenamiento.map(normalizar)
datos_prueba = datos_prueba.map(normalizar)

#Agregar a cache(usar memoria en lugar de disco, entrenamiento mas rapido)
datos_entrenamiento = datos_entrenamiento.cache()
datos_prueba = datos_prueba.cache()

In [ ]:
#Mostrar una imagen de los datos de prueba, de momento mostramos la primera
for imagen, etiqueta in datos_entrenamiento.take(1):
    break
imagen = imagen.numpy().reshape((28,28))
plt.figure()

plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
for i, (imagen, etiqueta) in enumerate(datos_entrenamiento.take(25)):
  imagen = imagen.numpy().reshape((28,28))
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(imagen, cmap= plt.cm.binary)
  plt.xlabel(nombres_clases[etiqueta])
plt.show()

In [49]:
#Creamos el modelo
modelo = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)) ,#1 - blanco y negro
    tf.keras.layers.Dense(50,activation= tf.nn.relu),
    tf.keras.layers.Dense(50,activation= tf.nn.relu),
    tf.keras.layers.Dense(10,activation= tf.nn.softmax) #para redes de clasificacion
])

In [51]:
#Compilar el Modelo
modelo.compile(
    optimizer="adam",
    loss= tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [50]:
num_entrenamiento = metadatos.splits['train'].num_examples
num_pruebas = metadatos.splits['test'].num_examples

In [52]:
TAMANO_LOTE = 32

datos_entrenamiento = datos_entrenamiento.repeat().shuffle(num_entrenamiento).batch(TAMANO_LOTE)
datos_prueba = datos_prueba.batch(TAMANO_LOTE)

In [ ]:
#Entrenar
historial = modelo.fit(datos_entrenamiento, epochs=5 , steps_per_epoch= math.ceil(num_entrenamiento / TAMANO_LOTE))

In [ ]:
plt.xlabel("# Epoca")
plt.ylabel("Magnitud de Perdida")
plt.plot(historial.history["loss"])

In [ ]:
#Pintar una cuadricula con varias predicciones, y marcar si fue correcta (azul) o incorrecta (roja)
for imagenes_prueba, etiquetas_prueba in datos_prueba.take(1):
  imagenes_prueba = imagenes_prueba.numpy()
  etiquetas_prueba = etiquetas_prueba.numpy()
  predicciones = modelo.predict(imagenes_prueba)

def graficar_imagen(i, arr_predicciones, etiquetas_reales, imagenes):
  arr_predicciones, etiqueta_real, img = arr_predicciones[i], etiquetas_reales[i], imagenes[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img[...,0], cmap=plt.cm.binary)

  etiqueta_prediccion = np.argmax(arr_predicciones)
  if etiqueta_prediccion == etiqueta_real:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(nombres_clases[etiqueta_prediccion],
                                100*np.max(arr_predicciones),
                                nombres_clases[etiqueta_real]),
                                color=color)

def graficar_valor_arreglo(i, arr_predicciones, etiqueta_real):
  arr_predicciones, etiqueta_real = arr_predicciones[i], etiqueta_real[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  grafica = plt.bar(range(10), arr_predicciones, color="#777777")
  plt.ylim([0, 1])
  etiqueta_prediccion = np.argmax(arr_predicciones)

  grafica[etiqueta_prediccion].set_color('red')
  grafica[etiqueta_real].set_color('blue')

filas = 5
columnas = 5
num_imagenes = filas*columnas
plt.figure(figsize=(2*2*columnas, 2*filas))
for i in range(num_imagenes):
  plt.subplot(filas, 2*columnas, 2*i+1)
  graficar_imagen(i, predicciones, etiquetas_prueba, imagenes_prueba)
  plt.subplot(filas, 2*columnas, 2*i+2)
  graficar_valor_arreglo(i, predicciones, etiquetas_prueba)